##### Required to use command line to run the code

In [ ]:
# the dataset's sources coming from this torrents
# https://academictorrents.com/details/56aa49f9653ba545f48df2e33679f014d2829c10

# decompress the zst data
# zstd -d --memory=2048MB RC_2010-04.zst (normally, we will replace the "RC_2010-04" into * for decompressing every zst file within the folder)

# use this code in command line for spliting the decompressed data into several small file then only load into python (need to apply to teenager and hiphophead decompressed data due to memoy problem while loading into python code)
# split -b 1048576000 teenagers_comments data_part_

# Loading the decompressed data

##### Function for loading the data (almost the clean code) into partition datasets and filtering them with important variables and timeframe

In [ ]:
# loading the decompressed data (comments dataset) into python code
import os
import re
import json
import pandas as pd

In [ ]:
def data_collection_comments(decompressed_data):
    base_path = "/home/haters/Downloads/temp_data"
    dir_path = os.path.join(base_path, decompressed_data, "Comments")
    output_dir = os.path.join(base_path, "loaded_data", decompressed_data, "comments")

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    file_name_comments = os.listdir(dir_path)
    comments_columns = ["body", "subreddit", "link_id", "retrieved_on"]
    start_date = pd.Timestamp("2023-01-01")
    end_date = pd.Timestamp("2023-12-31")

    chunksize = 1000000  # Define a suitable chunk size
    files_per_iteration = len(file_name_comments) // 5

    for iteration in range(1, 6):
        data_dicts_comments = []
        start_index = files_per_iteration * (iteration - 1)
        end_index = start_index + files_per_iteration if iteration < 5 else len(file_name_comments)
        
        for file_name in file_name_comments[start_index:end_index]:
            if 'Teenagers' in file_name or 'Music' in file_name or 'Hiphopheads' in file_name:
                print(f"Skipping file: {file_name}")
                continue
                
            file_path = os.path.join(dir_path, file_name)
            
            with open(file_path, 'r') as file:
                chunk_lines = []
    
                for line_number, line in enumerate(file, 1):
                    chunk_lines.append(line)
                    if line_number % chunksize == 0:
                        data_dicts_comments.extend(process_chunk(chunk_lines, comments_columns))
                        chunk_lines = []  # Reset for next chunk
    
                if chunk_lines:  # Process any remaining lines
                    data_dicts_comments.extend(process_chunk(chunk_lines, comments_columns))
                    
                print(f'completed file name::: {file_name}')
        save_data(data_dicts_comments, decompressed_data, output_dir, comments_columns, start_date, end_date, iteration, file_name)

def process_chunk(lines, columns):
    data_dicts = []
    for line in lines:
        try:
            # Load the line as a JSON object
            data_dict = json.loads(line)
            
            filtered_dict = {col: data_dict.get(col) for col in columns}  # Filter dict based on columns
            data_dicts.append(filtered_dict)
        except json.JSONDecodeError:
            print(f"Error parsing line: {line}")
    return data_dicts

def save_data(data_dicts, decompressed_data, output_dir, columns, start_date, end_date, iteration, file_name):
    loaded_dataset = pd.DataFrame(data_dicts)
    loaded_dataset = loaded_dataset[columns]
    loaded_dataset['retrieved_on'] = pd.to_datetime(loaded_dataset['retrieved_on'], unit='s')
    filtered_chunk = loaded_dataset[(loaded_dataset['retrieved_on'] >= start_date) & (loaded_dataset['retrieved_on'] <= end_date)]
    csv_path = os.path.join(output_dir, f"{decompressed_data}_test_{iteration}.csv")
    filtered_chunk.to_csv(csv_path, index=False)
    
    # Debugging output to monitor rows
    print(filtered_chunk.info())

In [ ]:
import os
import json
import pandas as pd

def data_collection_submissions(decompressed_data):
    # Set up base and output directories
    base_path = "/home/haters/Downloads/temp_data"
    dir_path = os.path.join(base_path, decompressed_data, "Submissions")
    output_dir = os.path.join(base_path, "loaded_data", decompressed_data, "submissions")

    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # List all files in the submissions directory
    file_name_submissions = os.listdir(dir_path)
    submissions_columns = ["selftext", "url", "title", "subreddit", "name", "permalink", "created_utc"]
    start_date = pd.Timestamp("2023-01-01")
    end_date = pd.Timestamp("2023-12-31")

    chunksize = 1000000  # Define a suitable chunk size
    files_per_iteration = len(file_name_submissions) // 5 # Split the files into 5 iterations

    for iteration in range(1, 6):
        data_dicts_submissions = []
        start_index = files_per_iteration * (iteration - 1)
        end_index = start_index + files_per_iteration if iteration < 5 else len(file_name_submissions)

        for file_name in file_name_submissions[start_index:end_index]:
            # Skip processing for specific subreddits
            if 'Teenagers' in file_name or 'Music' in file_name or 'Hiphopheads' in file_name:
                print(f"Skipping file: {file_name}")
                continue
            
            # Open and read the file line by line
            file_path = os.path.join(dir_path, file_name)
            with open(file_path, 'r') as file:
                chunk_lines = []

                for line_number, line in enumerate(file, 1):
                    chunk_lines.append(line)
                    if line_number % chunksize == 0:
                        data_dicts_submissions.extend(process_chunk(chunk_lines, submissions_columns))
                        save_data(data_dicts_submissions, decompressed_data, output_dir, submissions_columns, start_date, end_date, iteration, file_name)
                        data_dicts_submissions = []  # Reset for next chunk
                        
                # Process and save any remaining lines
                if chunk_lines: 
                    data_dicts_submissions.extend(process_chunk(chunk_lines, submissions_columns))
                    save_data(data_dicts_submissions, decompressed_data, output_dir, submissions_columns, start_date, end_date, iteration, file_name)

def process_chunk(lines, columns):
    data_dicts = []
    for line in lines:
        try:
            # Convert each line to a JSON object
            data_dict = json.loads(line)
            
            # Filter the JSON object based on required columns
            filtered_dict = {col: data_dict.get(col) for col in columns}
            data_dicts.append(filtered_dict)
        except json.JSONDecodeError:
            print(f"Error parsing line: {line}")
    return data_dicts

def save_data(data_dicts, decompressed_data, output_dir, columns, start_date, end_date, iteration, file_name):
    try:
        # Convert list of dict to a dataframe
        loaded_dataset = pd.DataFrame(data_dicts)
        loaded_dataset = loaded_dataset[columns]
        
        # Convert 'created_utc' to a datetime object to filtering
        loaded_dataset['created_utc'] = pd.to_datetime(loaded_dataset['created_utc'], unit='s')
        filtered_chunk = loaded_dataset[(loaded_dataset['created_utc'] >= start_date) & (loaded_dataset['created_utc'] <= end_date)]
        
        # Save the filtered data to a CSV file
        csv_path = os.path.join(output_dir, f"{decompressed_data}_submissions_{iteration}.csv")
        filtered_chunk.to_csv(csv_path, index=False)

        csv_path = os.path.join(output_dir, f"{decompressed_data}_submissions_sample_{iteration}.csv")
        filtered_chunk.to_csv(csv_path, index=False)
    
        # Debugging output to monitor rows
        print(filtered_chunk.info())
    except Exception as e: 
        print(f"Failed to process {file_name}: {str(e)}")

In [ ]:
# load those dataset from  into the python code

data_collection_comments("Male_Decompressed_Data")
data_collection_comments("Related_Decompressed_Data")
data_collection_comments("Female_Decompressed_Data")
data_collection_submissions("Male_Decompressed_Data")
data_collection_submissions("Female_Decompressed_Data")
data_collection_submissions("Related_Decompressed_Data")

## Merging the CSV datasets into a single file
Due to the memory issues, we split the loaded files into 4 or 5 parts and then mreged them into a single file

In [ ]:
def merge_datasets(decompressed_data, data_type="comments"):
    
    base_path = "/home/haters/Downloads"
    dir_path = os.path.join(base_path, "loaded_data", decompressed_data, data_type)
    
    files = [os.path.join(dir_path, f) for f in os.listdir(dir_path) if f.endswith('.csv')]

    dataframes = []
    
    # Read each file and append the DataFrame to the list
    for file in files:
        df = pd.read_csv(file)
        dataframes.append(df)

    # Save gthe merged DataFrame to a new csv file
    merged_df = pd.concat(dataframes, ignore_index=True)
    csv_path = os.path.join(dir_path, f"combined_{decompressed_data}.csv")
    
    print(f'csv path::: {csv_path}')
    
    return merged_df

In [ ]:
merge_co_female = merge_datasets('Female_Decompressed_Data', 'comments')
merge_co_male = merge_datasets('Male_Decompressed_Data', 'comments')
merge_co_related = merge_datasets('Related_Decompressed_Data', 'comments')
merge_sub_female = merge_datasets('Female_Decompressed_Data', 'submissions')
merge_sub_male = merge_datasets('Male_Decompressed_Data', 'submissions')
merge_sub_related = merge_datasets('Related_Decompressed_Data', 'submissions')

# Data Preprocessing

### Filtering data using the singer's name

In [ ]:
import os
import re
import pandas as pd

# Dictionary with artistname and their nicknames
artist_keywords = {
    ("Taylor Swift", "T-Swift", "TayTay", "Taylor", "Miss Americana"): "Taylor Swift",("SZA", "Solana Imani Row"): "SZA",
    ("Miley Cyrus", "Miley", "Hannah Montana"): "Miley Cyrus",
    ("New Jeans", "Minji", "Hanni", "Danielle New Jeans", "Haerin", "Hyein", "maknaes"): "New Jeans",
    ("Dua Lipa", "Dua", "Dula Peep"): "Dua Lipa",
    ("Olivia Rodrigo", "Liv"): "Olivia Rodrigo",
    ("Ariana Grande", "Ari", "Ariana", "Ms Grande"): "Ariana Grande",
    ("Billie Eilish", "Billie"): "Billie Eilish",
    ("Rihanna", "RiRi", "Badgalriri"): "Rihanna",
    ("Adele",): "Adele",
    ("Bad Bunny", "El Conejo Malo"): "Bad Bunny",
    ("The Weeknd", "Abel", "Abel Tesfaye"): "The Weeknd",
    ("Morgan Wallen",): "Morgan Wallen",
    ("Ed Sheeran", "Ginger Jesus", "Ed"): "Ed Sheeran",
    ("Drake", "Drizzy", "Champagne Papi", "Aubrey"): "Drake",
    ("Harry Styles",): "Harry Styles",
    ("Feid",): "Feid",
    ("Imagine Dragons", "Dan Reynolds", "Ben McKee", "Daniel Wayne Sermon", "Daniel Platzman"): "Imagine Dragons",
    ("Post Malone", "Posty"): "Post Malone",
    ("BTS", "Bangtan", "Bangtan Sonyeondan", "Tannies", "RM", "Jin", "Suga", "J-Hope", "Jimin", "V", "Jungkook"): "BTS"
}

# Function for checking each title against the dictionary of artist names
def find_artist(title):
    for keywords, artist_name in artist_keywords.items():
        for keyword in keywords:
            # Verwende Regex, um nur ganze Wörter zu matchen
            if re.search(r'\b' + re.escape(keyword) + r'\b', title, re.IGNORECASE):
                return artist_name
    return None

In [ ]:
# preprocess for female's related subreddits submission
related_submission_female = pd.read_csv('/home/haters/Downloads/temp_data/Related_Submissions_Decompressed_Data_female.csv')
related_submission_female_clean = related_submission_female.drop_duplicates(subset='permalink', keep='first')

# preprocess for male's related subreddits submission
related_submission_male = pd.read_csv('/home/haters/Downloads/temp_data/Related_Submissions_Decompressed_Data_male.csv')
related_submission_male_clean = related_submission_male.drop_duplicates(subset='permalink', keep='first')

print(related_submission_female_clean.head(10))
print(related_submission_male_clean.head(10))

### Merging submissions and comments

In [ ]:
def merging_related (submission,gender):
    
    # Lists of artists categorized by gender
    female_list = ["Taylor Swift", "SZA", "Miley Cyrus", "New Jeans", "Dua Lipa", "Olivia Rodrigo", "Ariana Grande", "Billie Eilish", "Rihanna", "Adele"]
    male_list = ["Bad Bunny", "The Weeknd", "Morgan Wallen", "Ed Sheeran", "Drake", "Harry Styles", "Feid", "Imagine Dragons", "Post Malone", "BTS"]
    
    # Identify the artist from the 'title' column in the submission dataset
    submission['artist'] = submission['title'].apply(find_artist)
    
    # Create a temporary ID from the 'title' column in the submission dataset
    submission['permalink'] = submission['permalink'].astype(str)
    submission['temp_id'] = submission['permalink'].str.split('/').str[4]
    submission_ids = submission["temp_id"].unique()
    
    comment_file_path = "/home/haters/Downloads/loaded_data/Related_Decompressed_Data/comments"
    file_name_comments = os.listdir(comment_file_path)
    
    all_data = pd.DataFrame() # Initialize an empty DataFrame
    
    for file_name in file_name_comments:
        if file_name.endswith('.csv'): # Process only CSV files
            file_path = os.path.join(comment_file_path, file_name)
            temp_df = pd.read_csv(file_path)
            temp_df["link_id"] = temp_df["link_id"].astype(str)
            temp_df["temp_id"] = temp_df["link_id"].str[3:] # Exact temporary ID from 'linked_id'
            temp_df = temp_df[temp_df['temp_id'].isin(submission_ids)] # Filter comments by submission IDs
            all_data = pd.concat([all_data, temp_df]) # Concatenate relevant comments
            
    # Merge submissions data with comments based on 'temp_id' to associate each comment with an artist        
    sub_data = submission[["temp_id", "artist"]]
    merged_dataset = pd.merge(all_data, sub_data, on='temp_id', how='inner')

    # Filter the merge dataset based on gender preference, excluding artists not matching the gender
    if gender == 'male':
        merged_dataset = merged_dataset[~merged_dataset['artist'].isin(female_list)]
    else:
        merged_dataset = merged_dataset[~merged_dataset['artist'].isin(male_list)]
        
    return merged_dataset

In [ ]:
related_submission_female = merging_related(related_submission_female_clean,'female')
related_submission_male = merging_related(related_submission_male_clean,'male')

### Convert the dataframe into csv file

In [ ]:
#Save the new submisisons with the artist

# Convert the data into csv file
related_submission_male.to_csv('related_submission_male.csv', index=False)
related_submission_female.to_csv('related_submission_female.csv', index=False)

## Checking the final result data

#### 1. Filtering data using the singer's name

In [ ]:
## Load data and combine them in Python / Adds a new column singer_name 

import pandas as pd
import os


class DataLoader:
    def __init__(self, data_name):
        
        self.data_name = data_name
        self.singer_map = {
            'female': {
                'taylor': 'Taylor Swift', 'swift': 'Taylor Swift', 'sza': 'SZA', 'miley': 'Miley Cyrus',
                'newjeans': 'New Jeans', 'maknaes': 'New Jeans', 'kimminji': 'New Jeans',
                'hanni': 'New Jeans', 'mojihye': 'New Jeans', 'dua': 'Dua Lipa',
                'olivia': 'Olivia Rodrigo', 'ariana': 'Ariana Grande', 'billie': 'Billie Eilish',
                'rihanna': 'Rihanna', 'adele': 'Adele'
            },
            'male': {
                'bunny': 'Bad Bunny', 'weeknd': 'The Weeknd', 'morgan': 'Morgan Wallen',
                'sheeran': 'Ed Sheeran', 'drake': 'Drake', 'harry': 'Harry Styles', 'larry': 'Harry Styles',
                'feid': 'Feid', 'dragon': 'Imagine Dragons', 'malone': 'Post Malone', 'bts': 'BTS', 
                'justin': 'Justin Bieber'
            }
        }
        
    def load_and_combine(self, data_type='comments'):
        # Load and combine all CSV files from a specified directory into a single DataFrame.
        
        file_path = f"/home/haters/Downloads/loaded_data/{self.data_name}/{data_type}"
        
        files = [os.path.join(file_path, file) for file in os.listdir(file_path) if file.endswith('.csv')]
        dataframes = []
        
        for file in files: 
            df = pd.read_csv(file)
            dataframes.append(df)
    
        # Generate a list of full paths to all CSV files in the directory
        files = [os.path.join(file_path, file) for file in os.listdir(file_path) if file.endswith('.csv')]        
        
        # Concatenate all DataFrames in the list into a single DataFrame
        combined_df = pd.concat(dataframes, ignore_index=True)
    
        ## Data Cleaning
        # Remove duplicate rows
        combined_df.drop_duplicates(inplace=True)
        # Handle missing values
        combined_df.dropna(inplace=True)
    
        if data_type == 'comments':
            combined_df = combined_df[(combined_df['body'] != '[deleted]') & (combined_df['body'] != '[removed]')]
        elif data_type == 'submissions':
            combined_df = combined_df[(combined_df['selftext'] != '[deleted]') & (combined_df['selftext'] != '[removed]')]
    
        return combined_df


    
    def add_singer_name(self, df, gender='female'): 
        # Adds a new column 'artist' to the DataFrame based on 'subreddit' column content.
        # Assigns singer's name based on keywords found in the 'subreddit' column.
    
        if gender in self.singer_map and 'subreddit' in df.columns:
            # Apply the mapping using a more robust function that checks string presence
            df['artist'] = df['subreddit'].apply(
                lambda x: next((name for key, name in self.singer_map[gender].items() if isinstance(x, str) and key in x.lower()), None)
            )
        else:
            raise ValueError("Invalid gender or 'subreddit' column missing")
    
        return df
    

**Load Dataset for male**

In [ ]:
## Load comments data

loader = DataLoader('Male_Decompressed_Data')
combined_male_comments = loader.load_and_combine('comments')
combined_male_comments = loader.add_singer_name(combined_male_comments, 'male')

print(f'rows: {combined_male_comments.shape[0]}')
combined_male_comments.head(10)

In [ ]:
## Load submissions data

loader = DataLoader('Male_Decompressed_Data')
combined_male_submissions = loader.load_and_combine('submissions')
combined_male_submissions = loader.add_singer_name(combined_male_submissions, 'female')
combined_male_submissions

**Load Dataset for female**

In [ ]:
## Load comments data

loader = DataLoader('Female_Decompressed_Data')
combined_female_comments = loader.load_and_combine('comments')
combined_female_comments = loader.add_singer_name(combined_female_comments, 'female')
combined_female_comments

In [ ]:
## Load submissions data

loader = DataLoader('Female_Decompressed_Data')
combined_female_submissions = loader.load_and_combine('submissions')
combined_female_submissions = loader.add_singer_name(combined_female_submissions, 'female')
combined_female_submissions

### Merging files into a single file for each gender: female and male

In [ ]:
def combine_datasets(csv_path, df, data_type, gender):
    # Loads a CSV file into a DataFrame, combines it with an existing DataFrame, and saves the combined DataFrame to a new CSV file.

    output_path = f"/home/haters/Downloads/loaded_data/combined_{gender}_{data_type}.csv"
    
    # Load the CSV file into a DataFrame
    new_df = pd.read_csv(csv_path)

    # Clean the data
    new_df.drop_duplicates(inplace=True)
    new_df.dropna(inplace=True)

    if data_type == 'comments':
        new_df = new_df[(new_df['body'] != '[deleted]') & (new_df['body'] != '[removed]')]
        
        count_specific_text = combined_male_comments['body'].str.contains(txt, na=False).sum()
        
    elif data_type == 'submissions':
        new_df = new_df[(new_df['selftext'] != '[deleted]') & (new_df['selftext'] != '[removed]')]
        
    # Combine the existing DataFrame with the new DataFrame
    combined_df = pd.concat([df, new_df], ignore_index=True)
     
    
    return combined_df

#### Submissions data

In [ ]:
# Combine the data into a single file(male)
path_male_com = '/home/haters/Downloads/temp_data/related_submission_male.csv'
male_submissions_df = combine_datasets(path_male_com, combined_male_submissions, 'submissions', 'male')

# Check for missing data
missing_artists = male_submissions_df[male_submissions_df['artist'].isna()]
print(f'missing rows(male submissions): {missing_artists.head()}')

# Check the combined data
print(f'total rows(male submissions): {male_submissions_df.shape[0]}')
male_submissions_df.head(10)

In [ ]:
# Combine the data into a single file(female)
path_female_com = '/home/haters/Downloads/temp_data/related_submission_female.csv'
female_submissions_df = combine_datasets(path_female_com, combined_female_submissions, 'submissions', 'female')

# Check for missing data
missing_artists = female_submissions_df[female_submissions_df['artist'].isna()]
print(f'missing rows(female submissions): {missing_artists.head()}')

# Check the combined data
print(f'total rows(female submissions): {female_submissions_df.shape[0]}')
female_submissions_df.head(10)

In [ ]:
# Convert the datatframe into csv file
male_submissions_df.to_csv("../loaded_data/male_submissions_outcome_final.csv")
female_submissions_df.to_csv("../loaded_data/female_submissions_outcome_final.csv")

#### Comments Data

In [ ]:
# Combine the data into a single file(male)
path_male_sub = '/home/haters/Downloads/temp_data/combine_male_related_subreddit.csv'
male_comments_df = combine_datasets(path_male_sub, combined_male_comments, 'comments', 'male')

# Check for missing data
missing_artists = male_comments_df[male_comments_df['artist'].isna()]
print(f'missing rows(male comments): {missing_artists.head()}')

# Check the combined data
print(f'total rows(male comments): {male_comments_df.shape[0]}')
male_comments_df.head(10)

In [ ]:
# Combine the data into a single file(female)
path_female_sub= '/home/haters/Downloads/temp_data/combine_female_related_subreddit.csv'
female_comments_df = combine_datasets(path_female_sub, combined_female_comments, 'comments', 'female')

# Check for missing data
missing_artists = female_comments_df[female_comments_df['artist'].isna()]
print(f'missing rows(fefmale comments): {missing_artists.head()}')

# Check the combined data
print(f'total rows(female comments): {female_comments_df.shape[0]}')
female_comments_df.head(10)

In [ ]:
# Convert the datatframe into csv file
male_comments_df.to_csv("../loaded_data/male_comments_outcome_final.csv")
female_comments_df.to_csv("../loaded_data/female_comments_outcome_final.csv")

## Text Preprocessing

In [ ]:
import re
import emoji
import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import WordPunctTokenizer
from nltk.stem import WordNetLemmatizer

In [ ]:
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')

In [ ]:
# Define Text Preprocessor class
class TextPreprocessor:
    def __init__(self):
        # Initialize the lemmatizer, stop words list, and tokenizer
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.tokenizer = WordPunctTokenizer()
        
        # List of singer names and their aliases to be removed from the text
        self.singer_names = [
            "Bad Bunny", "El Conejo Malo", "The Weeknd", "Abel", "Abel Tesfaye",
            "Morgan Wallen", "Ed Sheeran", "Ginger Jesus", "Ed", "Drake", 
            "Drizzy", "Champagne Papi", "Aubrey", "Harry Styles", "Feid", 
            "Imagine Dragons", "Dan Reynolds", "Ben McKee", "Daniel Wayne Sermon", 
            "Daniel Platz Platzman", "Post Malone", "Posty", "BTS", "Bangtan", 
            "Bangtan Sonyeondan", "Tannies", "RM", "Jin", "Suga", "J-Hope", 
            "Jimin", "V", "Jungkook", "harry", "justin bieber", "bieber", "justin", "namjoon", 
            "Taylor Swift" , "T-Swift" , "TayTay" , "Taylor" , "Miss Americana", "SZA", "Solana Imani Row", 
            "Miley Cyrus", "Miley", "Hannah Montana", "New Jeans", "Minji", "Hanni", "Danielle New Jeans", "Haerin", "Hyein", 
            "Dua Lipa", "Dua", "Dula Peep", "Olivia Rodrigo", "Liv", "Ariana Grande", "Ari", "Ariana", "Ms Grande", 
            "Billie Eilish", "Billie", "Rihanna", "RiRi", "Badgalriri", "Adele", "swiftie", "Oliia", "newjean", 
            "rodrigo", "rodrigos"
        ]
        # Convert all singer names to lowercase
        self.singer_names = [name.lower() for name in self.singer_names]

    def preprocess_text(self, text):
        text = text.lower() # Convert text into lowercase
        text = self.remove_singer_names(text) # Remove mentions of singer names
        text = emoji.demojize(text) # Convert emojis to text descriptions
        text = re.sub(r'[^\w\s]', '', text) # Remove punctuation and special characters
        text = re.sub(r'\d+', '', text)  # Remove digits
        tokens = self.tokenizer.tokenize(text) # Tokenize the text into words
        tokens = [word for word in tokens if word not in self.stop_words and word.isalnum()] # Remove stop words and non-alphanumeric tokens
        tokens = [self.lemmatize_token(word) for word in tokens] # Lemmatize each token to its root form
        
        # Return the processed tokens as a single string
        return " ".join(tokens)

    def remove_singer_names(self, text):
        # Remove occurrences of singer names with an empty string
        for name in self.singer_names:
            text = text.replace(name, '')
        
        # Additional specific text replacements
        text = text.replace('austin', 'album')
        text = text.replace('wts', 'want to sell')
        text = text.replace('merch', 'merchandise')
        
        return text

    def lemmatize_token(self, token):
        # Determine the part of POS tag for the token
        tag = self.get_wordnet_pos(nltk.pos_tag([token])[0][1])
        
        # Lemmatize the token based on its POS tag
        return self.lemmatizer.lemmatize(token, pos=tag) if tag else token

    @staticmethod
    def get_wordnet_pos(treebank_tag):
        # Converts treebank tag to wordnet tag
        if treebank_tag.startswith('J'):
            return wordnet.ADJ
        elif treebank_tag.startswith('V'):
            return wordnet.VERB
        elif treebank_tag.startswith('N'):
            return wordnet.NOUN
        elif treebank_tag.startswith('R'):
            return wordnet.ADV
        else:
            return None

In [ ]:
import pandas as pd

male_submissions = pd.read_csv("../loaded_data/male_submissions_outcome_final.csv")
female_submissions = pd.read_csv("../loaded_data/female_submissions_outcome_final.csv")

# Combine 'title' and 'selftext' into a new column 'combined_text'
male_submissions['combined_text'] = male_submissions['title'] + " " + male_submissions['selftext']
female_submissions['combined_text'] = female_submissions['title'] + " " + female_submissions['selftext']

In [ ]:
def remove_hyperlinks(text):
    return re.sub(r'http\S+|www\S+|https\S+', '', text, flags = re.MULTILINE)

male_submissions["combined_text"] = male_submissions["combined_text"].apply(remove_hyperlinks)
female_submissions["combined_text"] = female_submissions["combined_text"].apply(remove_hyperlinks)

In [ ]:
# Instantiate the TextPreprocessor class
preprocessor = TextPreprocessor()

def preprocess(df, data_type):
    
    if data_type == 'comments': # For comments, the text to preprocess is in the 'body' column
        text = 'body'
    elif data_type == 'submissions': # For submissions, the text to preprocess is in the 'combined_text' column
        text = 'combined_text'
        
    # Find the index of the text column where the new preprocessed column will be inserted
    col_index = df.columns.get_loc(text) + 1
    # Apply the text preprocessing
    preprocessed_text = df[text].apply(preprocessor.preprocess_text)
    # Insert the preprocessed text as a new column right after the original text column
    df.insert(col_index, 'preprocessed_txt', preprocessed_text)
    
    return df

In [ ]:
combined_male_submissions = preprocess(male_submissions, 'submissions')
combined_female_submissions = preprocess(female_submissions, 'submissions')